In [17]:
import numpy as np
import math
import GPy
import matplotlib.pyplot as plt
import seaborn

In [18]:
import numpy as np
from scipy.stats import qmc
from scipy.optimize import rosen  # Example (replace with Goldstein-Price)

# Goldstein-Price function (as defined in previous responses)
def goldstein_price(x):
    x1 = x[0]
    x2 = x[1]
    return (1 + (x1 + x2 + 1)**2 * (19 - 14*x1 + 3*x1**2 - 14*x2 + 6*x1*x2 + 3*x2**2)) * \
           (30 + (2*x1 - 3*x2)**2 * (18 - 32*x1 + 12*x1**2 + 48*x2 - 36*x1*x2 + 27*x2**2))



# Dimensionality of the problem
d = 2  # For Goldstein-Price (x1 and x2)

# Number of samples
n_samples = 32

# Create a Sobol sequence generator
sobol_engine = qmc.Sobol(d=d, scramble=False)  # scramble=True for better properties in high dimensions

# Generate Sobol points
sobol_points = sobol_engine.random_base2(m=int(np.log2(n_samples)))


# If n_samples is not a power of 2: (important!)
if not (n_samples != 0 and (n_samples & (n_samples - 1) == 0)): # Check if n_samples is not a power of 2
    sobol_points = sobol_engine.random(n=n_samples) #use random if not power of two


# Scale and shift to the Goldstein-Price domain [-2, 2]
domain_min = -2
domain_max = 2
scaled_sobol_points = domain_min + (domain_max - domain_min) * sobol_points

# Evaluate the function
function_values = np.array([goldstein_price(point) for point in scaled_sobol_points])



# Print the results (optional)
for i in range(n_samples):
    print(f"Point {scaled_sobol_points[i]}: Value {function_values[i]}")

# Example: Find the minimum (basic example)
min_index = np.argmin(function_values)
min_point = scaled_sobol_points[min_index]
min_value = function_values[min_index]

print("\nApproximate minimum (using Sobol sampling):")
print(f"Point: {min_point}")
print(f"Value: {min_value}")

Point [-2. -2.]: Value 24376.0
Point [0. 0.]: Value 600.0
Point [ 1. -1.]: Value 7100.0
Point [-1.  1.]: Value 87100.0
Point [-0.5 -0.5]: Value 32.6875
Point [1.5 1.5]: Value 9079.6875
Point [ 0.5 -1.5]: Value 657.6875
Point [-1.5  0.5]: Value 3082.6875
Point [-1.25 -0.75]: Value 1883.203125
Point [0.75 1.25]: Value 10647.328125
Point [ 1.75 -1.75]: Value 165496.484375
Point [-0.25  0.25]: Value 1933.984375
Point [-0.75 -1.25]: Value 945.703125
Point [1.25 0.75]: Value 864.828125
Point [ 0.25 -0.25]: Value 683.984375
Point [-1.75  1.75]: Value 594246.484375
Point [-1.625 -0.125]: Value 23405.27881717682
Point [0.375 1.875]: Value 117720.69141483307
Point [ 1.375 -1.125]: Value 31915.57746219635
Point [-0.625  0.875]: Value 48667.60840702057
Point [-0.125 -1.625]: Value 6274.446373939514
Point [1.875 0.375]: Value 207.68001651763916
Point [ 0.875 -0.625]: Value 571.5409784317017
Point [-1.125  1.375]: Value 251592.9017572403
Point [-1.375 -1.375]: Value 8176.620217323303
Point [0.625 0.

In [19]:
def generate_data():
    data = np.loadtxt('./../lda.csv', delimiter=",")
    sample = np.random.choice(data.shape[0], size = 32, replace=False)
    samp_data = data[sample]
    #X = samp_data[:, :-1]
    #y = samp_data[:, -1].reshape(-1, 1)
    X = data[:, :-2]
    y = data[:, -1].reshape(-1, 1)
    return X, y

def fit_gaussian_gpy(X, y):
    k = GPy.kern.RBF(input_dim=X.shape[1], variance=1.0, lengthscale=1.0)  # No need for separate constant kernel in the product
    mean = GPy.mappings.Constant(input_dim = X.shape[1], output_dim = 1, value = 0)
    model = GPy.models.GPRegression(X, y, k, noise_var=0.001**2, mean_function=mean)

    model.Gaussian_noise.variance.fix()
    #model.kern.lengthscale.constrain_bounded(1e-15, 1e15)
    #model.kern.variance.constrain_bounded(1e-20, 1e20)  # Constrain variance of RBF

    model.optimize_restarts(100, verbose=False)  

    print(model)  
    print(model.log_likelihood())
    #print("RBF variance:", model.kern.variance)
    #print("RBF lengthscale:", model.kern.lengthscale)
    #print("Constant Mean:", model.mean_function)
    print("BIC:", 3 * math.log(32) - 2 * model.log_likelihood())

In [20]:
fit_gaussian_gpy(scaled_sobol_points, function_values.reshape(-1, 1))

 /Users/anthonychiang/.pyenv/versions/3.11.5/lib/python3.11/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/anthonychiang/.pyenv/versions/3.11.5/lib/python3.11/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /Users/anthonychiang/.pyenv/versions/3.11.5/lib/python3.11/site-packages/GPy/kern/src/rbf.py:178: RuntimeWarning:invalid value encountered in multiply



Name : GP regression
Objective : 495.70835365268374
Number of Parameters : 4
Number of Optimization Parameters : 3
Updates : True
Parameters:
  GP_regression.           |               value  |  constraints  |  priors
  constmap.C               |   95102.97530261982  |               |        
  rbf.variance             |   1261565301.806181  |      +ve      |        
  rbf.lengthscale          |  0.5629718458820308  |      +ve      |        
  Gaussian_noise.variance  |               1e-06  |   +ve fixed   |        
-495.70835365268374
BIC: 1001.8139150137666


In [21]:
function_values_log = np.log(function_values)
fit_gaussian_gpy(scaled_sobol_points, function_values_log.reshape(-1, 1))


Name : GP regression
Objective : 64.20627488842976
Number of Parameters : 4
Number of Optimization Parameters : 3
Updates : True
Parameters:
  GP_regression.           |               value  |  constraints  |  priors
  constmap.C               |   9.161737105898371  |               |        
  rbf.variance             |   5.135553548767081  |      +ve      |        
  rbf.lengthscale          |  0.4696869272997326  |      +ve      |        
  Gaussian_noise.variance  |               1e-06  |   +ve fixed   |        
-64.20627488842976
BIC: 138.8097574852587
